# Chapter 4 - Exercises

> Author : Badr TAJINI - Large Language model (LLMs) - ESIEE 2024-2025

---

# Exercise 4.1: Parameters in the feed forward versus attention module

**Key Exercise Question: How do the parameter counts differ between the `feed-forward` neural network module and `multi-head attention` mechanism in our transformer architecture?**

*Methodological Approach:*
The investigation focuses on a systematic computational analysis of parameter allocation across two critical transformer neural network components:

1. **Feed-Forward Neural Network Module**
   - Characterization: Nonlinear transformation module
   - Primary computational function: Introducing network complexity and representational capacity
   - Parametric considerations: Linear transformation layers, activation functions

2. **Multi-Head Attention Mechanism**
   - Characterization: Contextual feature interaction module
   - Primary computational function: Capturing inter-token relational dynamics
   - Parametric considerations: Projection matrices, attention computation

*Analytical Objectives:*
- Quantify the exact number of trainable parameters in each architectural component
- Comparative assessment of parametric complexity
- Understand the relative computational resource allocation

*Theoretical Implications:*
- Insights into architectural parameter efficiency
- Empirical understanding of transformer module design
- Potential implications for model optimization and architectural design

*Computational Methodology:*
1. Enumerate parameters in `feed-forward` module
2. Enumerate parameters in `multi-head attention` module
3. Perform comparative statistical analysis
4. Interpret parametric distribution characteristics

*Recommended Investigative Approach:*
- Utilize precise computational tracing
- Consider layer-specific parameter counting
- Account for bias terms and weight matrices

In [3]:
import torch
import torch.nn as nn
from previous_labs import MultiHeadAttention

GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)

class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"], 
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        # Shortcut connection for feed forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        return x



class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        
        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        
        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

In [4]:
# FeedForward parameters
def calculate_feedforward_params(emb_dim):
    ff1_params = emb_dim * (4 * emb_dim)  
    ff1_bias = 4 * emb_dim  


    ff2_params = (4 * emb_dim) * emb_dim  
    ff2_bias = emb_dim  

    total_ff_params = ff1_params + ff1_bias + ff2_params + ff2_bias
    return total_ff_params

# MultiHeadAttention parameters : QKV projections + output projection
def calculate_mha_params(d_in, d_out, num_heads):

    mha_qkv_params = 3 * (d_in * d_out) # 3 because there are 3 projections and d_in * d_out parameters for each projection
    mha_out_params = d_out * d_out # output projection matrix

    total_mha_params = mha_qkv_params + mha_out_params
    return total_mha_params

#parameters calculation and comparaison
feedforward_params = calculate_feedforward_params(GPT_CONFIG_124M["emb_dim"])
mha_params = calculate_mha_params(GPT_CONFIG_124M["emb_dim"], GPT_CONFIG_124M["emb_dim"], GPT_CONFIG_124M["n_heads"])

print(f"FeedForward Parameters: {feedforward_params}")
print(f"Multi-Head Attention Parameters: {mha_params}")

FeedForward Parameters: 4722432
Multi-Head Attention Parameters: 2359296


We see that the FeedForward has almost twice more parameters than the Multi Head attention

# Exercise 4.2: Initialize larger GPT models

- **GPT2-small** (the 124M configuration we already implemented):
    - "emb_dim" = 768
    - "n_layers" = 12
    - "n_heads" = 12

- **GPT2-medium:**
    - "emb_dim" = 1024
    - "n_layers" = 24
    - "n_heads" = 16

- **GPT2-large:**
    - "emb_dim" = 1280
    - "n_layers" = 36
    - "n_heads" = 20

- **GPT2-XL:**
    - "emb_dim" = 1600
    - "n_layers" = 48
    - "n_heads" = 25

**Key Exercise Question: Can you systematically scale the GPT-2 model architecture from the small configuration to medium, large, and XL variants by exclusively modifying the configuration parameters?**

*Architectural Scaling Challenge:*
This exercise explores the methodological expansion of the GPT-2 model across different scales, demonstrating how architectural complexity can be incrementally increased through strategic parameter modifications.

*Model Variants to Implement:*
1. **GPT-2 Small (Current Implementation)**
   - Embedding Dimensions ("emb_dim"): 768
   - Transformer Blocks ("n_layers"): 12
   - Multi-Head Attention Heads ("n_heads"): 12

2. **GPT-2 Medium**
   - Embedding Dimensions ("emb_dim"): 1,024
   - Transformer Blocks ("n_layers"): 24
   - Multi-Head Attention Heads ("n_heads"): 16

3. **GPT-2 Large**
   - Embedding Dimensions ("emb_dim"): 1,280
   - Transformer Blocks ("n_layers"): 36
   - Multi-Head Attention Heads ("n_heads"): 20

4. **GPT-2 XL**
   - Embedding Dimensions ("emb_dim"): 1,600
   - Transformer Blocks ("n_layers"): 48
   - Multi-Head Attention Heads ("n_heads"): 25

*Methodological Constraints:*
- Modify only the configuration file
- Utilize the existing `GPTModel` class without code alterations
- Demonstrate parameter scaling capabilities
- Calculate total parameters for each model variant

**Bonus Challenge:**
**Compute the total number of trainable parameters for each model variant, highlighting the exponential growth in model complexity.**



In [ ]:
from gpt import GPTModel

GPT_CONFIG_2_Small = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

model = GPTModel(GPT_CONFIG_2_Small)

total_params = sum(p.numel() for p in model.parameters())
print("GPT Small : ", total_params)

GPT_CONFIG_2_Medium = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 1024,         # Embedding dimension
    "n_heads": 16,          # Number of attention heads
    "n_layers": 24,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

model = GPTModel(GPT_CONFIG_2_Medium)

total_params = sum(p.numel() for p in model.parameters())
print("GPT Medium : ", total_params)

GPT_CONFIG_2_Large = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 1280,         # Embedding dimension
    "n_heads": 20,          # Number of attention heads
    "n_layers": 36,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

model = GPTModel(GPT_CONFIG_2_Large)

total_params = sum(p.numel() for p in model.parameters())
print("GPT Large : ", total_params)


GPT Small :  163009536
GPT Medium :  406212608
GPT Large :  838220800


In [6]:
from gpt import GPTModel

GPT_CONFIG_2_XL = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 1600,         # Embedding dimension
    "n_heads": 25,          # Number of attention heads
    "n_layers": 48,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

model = GPTModel(GPT_CONFIG_2_XL)

total_params = sum(p.numel() for p in model.parameters())
print("GPT XL : ", total_params)

GPT XL :  1637792000


The compilation time to calculate parameters increase by 2 or 3 between each configuration

# Exercise 4.3: Using separate dropout parameters

**Key Exercise Question: How can we enhance the dropout configuration of the GPT model by implementing layer-specific dropout rates?**

*Architectural Dropout Refinement:*
The current implementation employs a uniform dropout rate across multiple model components, which presents an opportunity for more nuanced regularization strategies. This exercise challenges you to develop a more sophisticated approach to dropout implementation within neural network architectures.

*Dropout Localization:*
Three critical architectural components require distinct dropout configurations:
1. Embedding Layer
2. Shortcut (Residual) Connections
3. Multi-Head Attention Module

*Methodological Approach:*
You must modify the existing `GPT_CONFIG_124M` configuration to:
- Replace the monolithic `drop_rate` parameter
- Introduce a hierarchical dropout configuration
- Maintain the overall structural integrity of the model architecture

*Conceptual Challenge:*
The exercise requires a deep understanding of:
- Regularization techniques in neural network design
- The functional role of dropout in different architectural components
- Systematic configuration of model hyperparameters

In [7]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.5,       # change the dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}